## Data Visualization
First, we extract facial (3D) landmarks from a single participant, then we visualize those landmarks in 2 ways: 
* light weight (matplotlib 3D plot) 
* high fidelity (3D facial model using participant data and the FLAME model)

In [4]:
# general imports
import numpy as np
import csv
import matplotlib.pyplot as plt
import os
import sys
sys.path.append(os.path.abspath('../data'))

In [5]:
# Extract Facial Landmarks from CSV file
def extract_landmarks(path) -> np.ndarray:
    raw_data = [] # represens a single line of the file
    landmarks = np.zeros((68, 3))

    with open(path, 'r') as f:
        # if we wanted to extract multiple frames of landmarks, instead of just reading in the second line, read in the whole file or as many frames as desired
        next(f) # skips the first line
        raw_data = next(f) # read landmarks from a SINGLE (initial) frame
        if ('frame') in raw_data:
            raise Exception(f"Raw data contains file metadata; are you sure you skipped the header?")
        
        raw_data = raw_data.split(', ')
        raw_data = [float(x) for x in raw_data]
        raw_data = raw_data[4:]

        print(f'len: {len(raw_data)} / 3 = {len(raw_data)/3}')

        if len(raw_data) / 3 != 68:
            raise Exception(f"Data formatting error: {num_landmarks} landmarks found; expected 68. Check data to see if it's malformatted")

        # process raw landmark CSV data into numpy array
        for i in range(0, int(len(raw_data)/3)):
            x = raw_data[i]
            y = raw_data[68 + i]
            z = raw_data[68 + 68 + i]
            landmarks[i] = [x, y, z]
    
    return landmarks

In [7]:
# lightweight implementation using MatplotLib 3D Scatterplot
def plot_3D(landmarks: np.ndarray):
    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')
    ax.view_init(elev=-90, azim=-90, roll=0)

    xs, ys, zs = landmarks[:,0], landmarks[:,1], landmarks[:,2]

    ax.scatter(xs, ys, zs)
    ax.set(xticklabels=[],
       yticklabels=[],
       zticklabels=[])
    plt.title('3D facial landmarks')
    plt.show()




if __name__ == "__main__":
    print(f'running align-faces...')
    landmarks_A = extract_landmarks("../data/300_P/300_CLNF_features3D.txt")
    # print(f'dims: {landmarks_A.shape} \n{landmarks_A}')
    plot_3D(landmarks_A)

running align-faces...


FileNotFoundError: [Errno 2] No such file or directory: '../data/300_P/300_CLNF_features3D.txt'

In [ ]:
# High Fidelity Implementation using FLAME Model
# this downsizes 68 facial landmarks to 51 (which the FLAME model requires)
# then use the 51 facial landmarks as input to the flame model
# TODO: fill this in